<a href="https://colab.research.google.com/github/kaustubhpatil2611/ai_assignments/blob/master/AI_LAB_EXAM_WEB_CRAWLER_T204045.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**AI PRACTICAL EXAM**

*Kaustubh Patil*

*SeatNo:T204045*

*PRN:0120180264*

*Roll No:059*

Write a program to build index of Crawlers in Search Engines using Depth First Search, start from source page and follow all links from source and keep doing same. (Assume suitable data as applicable)

In [6]:
#import all the required libraries
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import urllib#library for openning urls
import nltk
from bs4 import BeautifulSoup#DOM parsing library
import re
sword = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#class for maintaing url and their index values
class searchIndex:
  def __init__(self,url,indexValue):
    self.url=url
    self.indexValue=indexValue


#function to get html content from target url
def get_html(url):
  try:
    html_page=urllib.request.urlopen(url)#get url contents
    soup=BeautifulSoup(html_page)#parse the contents
    
  except Exception as e:#if bad response return error
    return 'error'

  return soup

#get tokens from searched query
def get_search_tokens(query):
  tokens= [t for t in query.split()]#split sentences to words
  clean_tokens= []
  for token in tokens:
    if token.lower() not in sword:
      clean_tokens.append(token.lower())
  return clean_tokens

#get high frequency key words
def get_searches_tokens(url):
  response = urllib.request.urlopen(url)
  html = response.read()
  soup = BeautifulSoup(html,"html.parser")#parse html content
  text = soup.get_text(strip=True)#get only text
  tokens = [t for t in text.split()]
  sword = stopwords.words('english')
  symbols=['{','}',';','|',':','=','using','given']
  sword=sword+symbols
  #get processed text
  clean_tokens = []
  for token in tokens:
      if token.lower() not in sword:
          clean_tokens.append(token.lower())

  #calculate word frequencies
  freq=nltk.FreqDist(clean_tokens)
  final_tokens=[m for m,n in freq.items() if n>=15 and len(m)>3 ]#consider words with frequency higher than 14 and word length more than 3
  return final_tokens

#function to get count of matched keywords in the page
def get_indexValue(url,searchKeys):
  response=urllib.request.urlopen(url)
  html=response.read()
  soup=BeautifulSoup(html,"html.parser")
  text=soup.get_text(strip=True)
  tokens = [t for t in text.split()]
  clean_token = []
  count=0
  for token in tokens:
      if token.lower() not in sword:
          clean_token.append(token.lower())
  for key in clean_token:
    if key in searchKeys:#check if key is present in the page
      count=count+1
  return count


#key function used for sorting class searchIndex
def get_me_key(obj):
  return obj.indexValue



In [67]:
#function for search strategy for web crawler in bfs or dfs
def webcrawler_dfs_bfs(source_url,searchKeys,searchStrategy):
  url_frontier_stack=list()#frontier stack /queue::dfs/bfs
  url_frontier_stack.append(source_url)#push the source url
  searchResults=list()#list to keep track of url and their index score
  visited_urls=set()#set to track the visited nodes
  t=0
  similar=[]#to keep track of same urls in same page
  if searchStrategy==1:#if dfs
    while t<50:
      node_url=url_frontier_stack.pop()#pop from top of stack
      visited_urls.add(node_url)#add current url node as visited
      soup=get_html(node_url)#get html contents
      if soup=='error':
        searchResults.append(searchIndex(node_url,-1000))#if bad response index the page with least score
      else:
        value=get_indexValue(node_url,searchKeys)
        if value>10:#else add url and its index score if relevant page
          searchResults.append(searchIndex(node_url,value)) 
          for link in soup.findAll('a', attrs={'href': re.compile("^https://")}):#find all child urls and add to frontier if not visited
            if link.get('href') not in similar:
              if link.get('href') not in visited_urls:
                similar.append(link.get('href'))
                url_frontier_stack.append(link.get('href'))
          t=t+1
        else:
          searchResults.append(searchIndex(node_url,0))
  else:#else bfs
    while t<50:
      node_url=url_frontier_stack.pop(0)#pop from front
      visited_urls.add(node_url)
      soup=get_html(node_url)
      if soup=='error':
        searchResults.append(searchIndex(node_url,-1000))
      else:
        value=get_indexValue(node_url,searchKeys)
        if value>10:
          searchResults.append(searchIndex(node_url,value)) 
          for link in soup.findAll('a', attrs={'href': re.compile("^https://")}):
            if link.get('href') not in similar:
              if link.get('href') not in visited_urls:
                similar.append(link.get('href'))
                url_frontier_stack.append(link.get('href'))
          t=t+1
        else:
          searchResults.append(searchIndex(node_url,0))
    

  searchResults.sort(key=get_me_key,reverse=True)#get the final searchResult of crawling url::indexValue in best to worst order

  print('Here are the best search results available:\n')

  count=0
  for result in searchResults:#print top 20 results
    if count==20:
      break
    print(result.url,' value:',result.indexValue)
    count=count+1


In [69]:
#main program starts here
searchKeys=set()
source_url=input('Enter the source URL:')#get source url
query=input("Enter the search...")#get search key word
#update searchKeys as per query and source url
searchKeys.add(query)
searchKeys.update(get_search_tokens(query))
searchKeys.update(get_searches_tokens(source_url))
#input your strategy
strategy=input("Type BFS or DFS as per your strategy:")
if strategy=='BFS' or strategy=='bfs':
  searchStrategy=0
elif strategy=='DFS' or strategy=='dfs':
  searchStrategy=1
else:
  print("Entering default mode")
  searchStrategy=0
#webcrawler crawling...
webcrawler_dfs_bfs(source_url,searchKeys,searchStrategy)

Enter the source URL:https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/
Enter the search...breadth first search
Type BFS or DFS as per your strategy:bfs


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Here are the best search results available:

https://www.geeksforgeeks.org/prims-minimum-spanning-tree-mst-greedy-algo-5/?ref=leftbar-rightbar  value: 120
https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/  value: 93
https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/?ref=lbp  value: 93
https://www.geeksforgeeks.org/applications-of-breadth-first-traversal/?ref=rp  value: 93
https://www.geeksforgeeks.org/applications-of-breadth-first-traversal/  value: 92
https://www.geeksforgeeks.org/depth-first-search-or-dfs-for-a-graph/?ref=leftbar-rightbar  value: 89
https://www.geeksforgeeks.org/depth-first-traversal-for-a-graph/  value: 89
https://www.geeksforgeeks.org/depth-first-search-or-dfs-for-a-graph/?ref=rp  value: 89
https://www.geeksforgeeks.org/dijkstras-shortest-path-algorithm-greedy-algo-7/?ref=leftbar-rightbar  value: 84
https://www.geeksforgeeks.org/check-given-directed-graph-strongly-connected-set-2-kosaraju-using-bfs/?ref=rp  value: 77
https:/

In [70]:
#main program starts here
searchKeys=set()
source_url=input('Enter the source URL:')
query=input("Enter the search...")
searchKeys.add(query)
searchKeys.update(get_search_tokens(query))
searchKeys.update(get_searches_tokens(source_url))
strategy=input("Type BFS or DFS as per your strategy:")
if strategy=='BFS' or strategy=='bfs':
  searchStrategy=0
elif strategy=='DFS' or strategy=='dfs':
  searchStrategy=1
else:
  print("Entering default mode")
  searchStrategy=0
webcrawler_dfs_bfs(source_url,searchKeys,searchStrategy)

Enter the source URL:https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/
Enter the search...breadth first search
Type BFS or DFS as per your strategy:dfs
Here are the best search results available:

https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/  value: 93
https://www.geeksforgeeks.org/dijkstras-algorithm-for-adjacency-list-representation-greedy-algo-8/?ref=leftbar-rightbar  value: 66
https://www.geeksforgeeks.org/ford-fulkerson-algorithm-for-maximum-flow-problem/?ref=leftbar-rightbar  value: 51
https://www.geeksforgeeks.org/jump-search/?ref=leftbar-rightbar  value: 38
https://www.geeksforgeeks.org/binary-search-bisect-in-python/?ref=leftbar-rightbar  value: 35
https://www.geeksforgeeks.org/linear-search-vs-binary-search/  value: 35
https://www.geeksforgeeks.org/difference-between-multi-level-queue-scheduling-mlq-and-first-come-first-served-fcfs/  value: 32
https://www.geeksforgeeks.org/linear-search/?ref=leftbar-rightbar  value: 28
https://ww

In [79]:
#function for search strategy for web crawler in iterative deepening search method
def webcrawler_ids(node_url,searchKeys):
  
  #print(len(visited_urls))
  for i in range(4):
    visited_urls=set()
    searchResults=list()
    visited_urls,searchResults=webcrawler_dls(node_url,i,visited_urls,searchResults,searchKeys)#call crawler search for different depths
    #print(i,len(visited_urls))

  searchResults.sort(key=get_me_key,reverse=True)#sort the result as per best to worst order
  #print(len(visited_urls))
  print('Here are the best search results available:\n')
  count=0
  for result in searchResults:
    if count==20:
      break
    print(result.url,' value:',result.indexValue)
    count=count+1
  return True

def webcrawler_dls(node_url,limit,visited_urls,searchResults,searchKeys):
  visited_urls.add(node_url)#add current node url as visited
  if limit==0:#if limit ended return
    return visited_urls,searchResults
  soup=get_html(node_url)
  if soup=='error':#if bad response cut the branch
    searchResults.append(searchIndex(node_url,-1000))
  else:
    value=get_indexValue(node_url,searchKeys)
    if value>10:#if good keywords frequency push to frontier
      searchResults.append(searchIndex(node_url,value))#and add their result
      similar=[] 
      for link in soup.findAll('a', attrs={'href': re.compile("^https://")}):#find all child url
        if link.get('href') not in similar:
          if link.get('href') not in visited_urls:#if not visited
            #print(link.get('href'))
            similar.append(link.get('href'))
            visited_urls,searchResults=dls(link.get('href'),limit-1,visited_urls,searchResults,searchKeys)#then call search with this child as source url and one less depth limit
    else:
      searchResults.append(searchIndex(node_url,0))#if not useful search add index value as 0
      return visited_urls,searchResults
  return visited_urls,searchResults

In [80]:
#main program starts here
searchKeys=set()
source_url=input('Enter the source URL:')
query=input("Enter the search...")
searchKeys.add(query)
searchKeys.update(get_search_tokens(query))
searchKeys.update(get_searches_tokens(source_url))
webcrawler_ids(source_url,searchKeys)

Enter the source URL:https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/
Enter the search...breadth first search


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Here are the best search results available:

https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/  value: 93
https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/?ref=leftbar-rightbar  value: 93
https://www.geeksforgeeks.org/breadth-first-traversal-for-a-graph/  value: 93
https://www.geeksforgeeks.org/connectivity-in-a-directed-graph/?ref=rp  value: 93
https://www.geeksforgeeks.org/depth-first-search-or-dfs-for-a-graph/?ref=leftbar-rightbar  value: 89
https://www.geeksforgeeks.org/graph-and-its-representations/  value: 62
https://www.geeksforgeeks.org/archives/11644  value: 49
https://www.geeksforgeeks.org/applications-of-depth-first-search/?ref=rp  value: 49
https://www.geeksforgeeks.org/print-the-lexicographically-smallest-dfs-of-the-graph-starting-from-1/?ref=rp  value: 33
https://www.geeksforgeeks.org/archives/618  value: 23
https://www.geeksforgeeks.org/position-of-rightmost-set-bit/?ref=lbp  value: 19
https://www.geeksforgeeks.org/analysis-of-al

True

In [41]:
#testing purpose
#main program starts here
searchKeys=set()
source_url=input('Enter the source URL:')
query=input("Enter the search...")
searchKeys.add(query)
searchKeys.update(get_search_tokens(query))
searchKeys.update(get_searches_tokens(source_url))
webcrawler_ids(source_url,searchKeys)

0 1
1 200


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


2 838
3 1636


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


4 7622
7622
Here are the best search results available:

https://www.geeksforgeeks.org/applications-of-breadth-first-traversal/  value: 102
https://www.geeksforgeeks.org/applications-of-breadth-first-traversal/?ref=lbp  value: 102
https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/  value: 87
https://www.geeksforgeeks.org/breadth-first-traversal-for-a-graph  value: 87
https://www.geeksforgeeks.org/level-order-tree-traversal/  value: 71
https://www.geeksforgeeks.org/level-order-tree-traversal/?ref=lbp  value: 71
https://www.geeksforgeeks.org/tree-traversals-inorder-preorder-and-postorder/?ref=lbp  value: 69
https://www.geeksforgeeks.org/queue-using-stacks/?ref=leftbar-rightbar  value: 64
https://www.geeksforgeeks.org/find-the-minimum-element-in-a-binary-search-tree/  value: 52
https://www.geeksforgeeks.org/bipartite-graph/  value: 48
https://www.geeksforgeeks.org/find-the-first-n-pure-numbers/?ref=rp  value: 45
https://www.geeksforgeeks.org/level-order-traversal-line-

True

In [ ]:

#Depth First Search:(Expanded URL Limits=50)
'''
Here are the best search results available:

https://en.wikipedia.org/wiki/Natural_language_processing  value: 148
https://philpapers.org/go.pl?id=BRANS&proxyId=none&u=http%3A%2F%2Fplato.stanford.edu%2Fentries%2Fnatural-selection%2F  value: 107
https://philpapers.org/go.pl?id=BRANS&proxyId=&u=http%3A%2F%2Fplato.stanford.edu%2Fentries%2Fnatural-selection%2F  value: 107
https://philpapers.org/browse/natural-language-processing  value: 82
https://plato.stanford.edu/archives/fall2019/entries/natural-selection/  value: 72
https://philpapers.org/rec/PINNSA  value: 28
https://philpapers.org/rec/CHRTNB  value: 24
https://philpapers.org/rec/PICTAM  value: 21
https://philpapers.org/asearch.pl?pub=139  value: 20
https://philpapers.org/sep/natural-selection/  value: 18
https://philpapers.org/go.pl?id=BARIPI-5&proxyId=&u=http%3A%2F%2Fdx.doi.org%2F10.1007%2Fs10539-020-9735-6  value: 16
https://doi.org/10.1007/s10539-020-9735-6  value: 16
https://philpapers.org/rec/SPIDAE  value: 15
https://philpapers.org/go.pl?id=BARILP&proxyId=&u=http%3A%2F%2Fdx.doi.org%2F10.1017%2FS0140525X04220052  value: 14
https://doi.org/10.1017/S0140525X04220052  value: 14
https://philpapers.org/rec/BARILP  value: 13
https://philpapers.org/rec/SHITNO  value: 13
https://doi.org/10.1073%2Fpnas.1620732114  value: 13
https://www.wikidata.org/wiki/Special:EntityPage/Q30642#sitelinks-wikipedia  value: 12
https://www.wikidata.org/w/index.php?title=Q30642&oldid=1303758695  value: 12

Enter the source URL:https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/
Enter the search...breadth first search
Type BFS or DFS as per your strategy:dfs
Here are the best search results available:

https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/  value: 93
https://www.geeksforgeeks.org/dijkstras-algorithm-for-adjacency-list-representation-greedy-algo-8/?ref=leftbar-rightbar  value: 66
https://www.geeksforgeeks.org/ford-fulkerson-algorithm-for-maximum-flow-problem/?ref=leftbar-rightbar  value: 51
https://www.geeksforgeeks.org/jump-search/?ref=leftbar-rightbar  value: 38
https://www.geeksforgeeks.org/binary-search-bisect-in-python/?ref=leftbar-rightbar  value: 35
https://www.geeksforgeeks.org/linear-search-vs-binary-search/  value: 35
https://www.geeksforgeeks.org/difference-between-multi-level-queue-scheduling-mlq-and-first-come-first-served-fcfs/  value: 32
https://www.geeksforgeeks.org/linear-search/?ref=leftbar-rightbar  value: 28
https://www.geeksforgeeks.org/difference-between-multi-level-queue-scheduling-mlq-and-shortest-job-first/  value: 27
https://www.geeksforgeeks.org/sort-first-k-values-in-ascending-order-and-remaining-n-k-values-in-descending-order/?ref=rp  value: 26
https://www.geeksforgeeks.org/program-for-fcfs-cpu-scheduling-set-1/?ref=leftbar-rightbar  value: 26
https://www.geeksforgeeks.org/program-for-fcfs-cpu-scheduling-set-1/  value: 26
https://www.geeksforgeeks.org/difference-between-multi-level-queue-scheduling-mlq-and-shortest-job-first/?ref=rp  value: 26
https://www.geeksforgeeks.org/searching-map-using-stdmap-functions-c/?ref=leftbar-rightbar  value: 24
https://www.geeksforgeeks.org/cpu-scheduling-in-operating-systems/?ref=leftbar-rightbar  value: 24
https://www.geeksforgeeks.org/program-for-shortest-job-first-or-sjf-cpu-scheduling-set-1-non-preemptive/  value: 24
https://www.geeksforgeeks.org/sort-array-according-order-defined-another-array/?ref=leftbar-rightbar  value: 23
https://www.geeksforgeeks.org/difference-between-prims-and-kruskals-algorithm-for-mst/?ref=leftbar-rightbar  value: 21
https://www.geeksforgeeks.org/delete-operation-in-b-tree/?ref=leftbar-rightbar  value: 21
https://www.geeksforgeeks.org/delete-operation-in-b-tree/  value: 21

'''


#Breadth First Search:(Expanded URL Limits=50)
'''
Enter the source URL:https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/
Enter the search...breadth first search
Type BFS or DFS as per your strategy:bfs
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Here are the best search results available:

https://www.geeksforgeeks.org/prims-minimum-spanning-tree-mst-greedy-algo-5/?ref=leftbar-rightbar  value: 120
https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/  value: 93
https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/?ref=lbp  value: 93
https://www.geeksforgeeks.org/applications-of-breadth-first-traversal/?ref=rp  value: 93
https://www.geeksforgeeks.org/applications-of-breadth-first-traversal/  value: 92
https://www.geeksforgeeks.org/depth-first-search-or-dfs-for-a-graph/?ref=leftbar-rightbar  value: 89
https://www.geeksforgeeks.org/depth-first-traversal-for-a-graph/  value: 89
https://www.geeksforgeeks.org/depth-first-search-or-dfs-for-a-graph/?ref=rp  value: 89
https://www.geeksforgeeks.org/dijkstras-shortest-path-algorithm-greedy-algo-7/?ref=leftbar-rightbar  value: 84
https://www.geeksforgeeks.org/check-given-directed-graph-strongly-connected-set-2-kosaraju-using-bfs/?ref=rp  value: 77
https://www.geeksforgeeks.org/graph-and-its-representations/?ref=leftbar-rightbar  value: 62
https://www.geeksforgeeks.org/graph-data-structure-and-algorithms/  value: 59
https://www.geeksforgeeks.org/detect-cycle-in-an-undirected-graph-using-bfs/?ref=rp  value: 58
https://www.geeksforgeeks.org/union-find/?ref=leftbar-rightbar  value: 56
https://www.geeksforgeeks.org/topological-sorting/?ref=leftbar-rightbar  value: 55
https://www.geeksforgeeks.org/best-first-search-informed-search/?ref=rp  value: 55
https://www.geeksforgeeks.org/detect-cycle-in-a-graph/?ref=leftbar-rightbar  value: 54
https://www.geeksforgeeks.org/detect-cycle-in-a-directed-graph-using-bfs/?ref=rp  value: 52
https://www.geeksforgeeks.org/iterative-deepening-searchids-iterative-deepening-depth-first-searchiddfs/?ref=rp  value: 51
https://www.geeksforgeeks.org/applications-of-depth-first-search/  value: 49

'''

#Iterative deepening search/Depth Limited Search: Depth-Limit=3
'''
Enter the source URL:https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/
Enter the search...breadth first search
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Here are the best search results available:

https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/  value: 93
https://www.geeksforgeeks.org/breadth-first-search-or-bfs-for-a-graph/?ref=leftbar-rightbar  value: 93
https://www.geeksforgeeks.org/breadth-first-traversal-for-a-graph/  value: 93
https://www.geeksforgeeks.org/connectivity-in-a-directed-graph/?ref=rp  value: 93
https://www.geeksforgeeks.org/depth-first-search-or-dfs-for-a-graph/?ref=leftbar-rightbar  value: 89
https://www.geeksforgeeks.org/graph-and-its-representations/  value: 62
https://www.geeksforgeeks.org/archives/11644  value: 49
https://www.geeksforgeeks.org/applications-of-depth-first-search/?ref=rp  value: 49
https://www.geeksforgeeks.org/print-the-lexicographically-smallest-dfs-of-the-graph-starting-from-1/?ref=rp  value: 33
https://www.geeksforgeeks.org/archives/618  value: 23
https://www.geeksforgeeks.org/position-of-rightmost-set-bit/?ref=lbp  value: 19
https://www.geeksforgeeks.org/analysis-of-algorithms-set-1-asymptotic-analysis/  value: 18
https://practice.geeksforgeeks.org/problems/depth-first-traversal-for-a-graph/1  value: 9
https://www.geeksforgeeks.org/system-design-of-uber-app-uber-system-architecture/  value: 7
https://www.geeksforgeeks.org/tag/morgan-stanley/  value: 6
https://www.geeksforgeeks.org/medium/  value: 5
https://www.geeksforgeeks.org/easy/  value: 5
https://www.geeksforgeeks.org/vmware-interview-experience-on-campus-3/  value: 4
https://www.geeksforgeeks.org/count-total-bits-number/  value: 4
https://www.geeksforgeeks.org/n-queen-problem-backtracking-3/?ref=lbp  value: 4
True
'''